# Process Data
* Varous data cleaning tasks

In [ ]:
import datetime
import glob
import os
import shutil
import subprocess
import time

import numpy as np
import papermill as pm

In [ ]:
data_path = "../../data/papermill/process_data"
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

# Helpers for spawning notebooks

In [ ]:
def run_background_notebook(input, output, parameters):
    cmdlist = [
        "papermill",
        input,
        output,
    ]
    for k, v in parameters.items():
        cmdlist += ["-p", k, str(v)]
    return subprocess.Popen(
        cmdlist, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
    )

In [ ]:
def run_notebook(input, output=None, parameters=None, background=False):
    if output is None:
        output = input
    input = f"{input}.ipynb"
    output = os.path.join(data_path, f"{output}.ipynb")
    parameters = parameters if parameters is not None else dict()
    if background:
        return run_background_notebook(input, output, parameters)
    else:
        return pm.execute_notebook(input, output, parameters=parameters)

In [ ]:
def run_notebooks(script):
    for media in ["manga", "anime"]:
        run_notebook(script, f"{script}{media.capitalize()}", {"media": media})

In [ ]:
def run_parts(script, parts, params=None):
    start_time = time.time()
    print(f"spawning {len(parts)} processes")
    procs = []
    for part in parts:
        p = {"part": part}
        if params is not None:
            p |= params
        procs.append(
            run_notebook(
                script,
                output=f"{script}.{'.'.join(str(x) for x in p.values())}",
                parameters=p,
                background=True,
            )
        )
        time.sleep(1)
    for p in procs:
        p.wait()
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    print(f"processes finished in {int(round(time.time() - start_time))} seconds")

# Run notebooks

In [ ]:
num_parts = 16

In [ ]:
run_notebooks("ProcessMedia")

In [ ]:
for x in np.array_split(range(num_parts), 4):
    run_parts("PruneMediaLists", x)

In [ ]:
run_notebook("MapUids");

In [ ]:
run_parts("ProcessMediaLists", range(num_parts))

In [ ]:
for media in ["manga", "anime"]:
    for target_media in ["manga", "anime"]:
        run_notebook(
            "RelatedMedia",
            (
                "RelatedMedia"
                + f"{media.capitalize()}{target_media.capitalize()}"
                + ".ipynb"
            ),
            {"media": media, "target_media": target_media},
        )

In [ ]:
run_parts("KnowledgeCutoff", range(num_parts))

In [ ]:
for media in ["anime", "manga"]:
    for chunk in np.array_split(list(range(num_parts)), 4):
        run_parts("GenerateSplits", chunk, {"media": media})

In [ ]:
run_notebook("CompressSplits")

In [ ]:
# reclaim disk space
shutil.rmtree("../../data/raw_data")
for f in glob.glob("../../data/splits/*.h5"):
    os.remove(f)
for f in glob.glob("../../data/processed_data/*prune*.csv"):
    os.remove(f)
for f in glob.glob("../../data/processed_data/user_*_list.*.csv"):
    os.remove(f)

In [ ]:
print("Success!")